In [1]:
# Imports
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import pretty_midi
import librosa
import mir_eval
# import mir_eval.display
#import tables
import IPython.display
import os
import json
import csv


# Local path constants
DATA_PATH = 'data'
RESULTS_PATH = '/Users/joro/Downloads/'
# Path to the file match_scores.json distributed with the LMD
SCORE_FILE = os.path.join(RESULTS_PATH, 'match_scores.json')

# Utility functions for retrieving paths
def msd_id_to_dirs(msd_id):
    """Given an MSD ID, generate the path prefix.
    E.g. TRABCD12345678 -> A/B/C/TRABCD12345678"""
    return os.path.join(msd_id[2], msd_id[3], msd_id[4], msd_id)

def msd_id_to_mp3(msd_id):
    """Given an MSD ID, return the path to the corresponding mp3"""
    return os.path.join(DATA_PATH, 'msd', 'mp3',
                        msd_id_to_dirs(msd_id) + '.mp3')

def msd_id_to_h5(h5):
    """Given an MSD ID, return the path to the corresponding h5"""
    return os.path.join(RESULTS_PATH, 'lmd_matched_h5',
                        msd_id_to_dirs(msd_id) + '.h5')

def get_midi_path(msd_id, midi_md5, kind):
    """Given an MSD ID and MIDI MD5, return path to a MIDI file.
    kind should be one of 'matched' or 'aligned'. """
    return os.path.join(RESULTS_PATH, 'lmd_{}'.format(kind),
                        msd_id_to_dirs(msd_id), midi_md5 + '.mid')

In [3]:
with open(SCORE_FILE) as f:
    scores = json.load(f)
    
    # Grab an MSD ID and its dictionary of matches
# msd_id = 'TRALLSG128F425A685'
msd_id = 'TROOCBX128F429648A'
matches = scores[msd_id]

######### get MIDI from the matches. 

# grab random one
midi_md5, score = matches.popitem()
print '  {} with confidence score {}'.format(midi_md5, score)

# or check score confidence values
# for midi_md5, score in scores[msd_id].items():
#     print '  {} with confidence score {}'.format(midi_md5, score)

    # Construct the path to the aligned MIDI
aligned_midi_path = get_midi_path(msd_id, midi_md5, 'aligned')
    # Load/parse the MIDI file with pretty_midi
pm = pretty_midi.PrettyMIDI(aligned_midi_path)
    # Look for a MIDI file which has lyric and key signature change events


  76fec7030a2f504186aae20d4983929f with confidence score 0.650326442104


In [6]:
intervals = np.array([[note.start, note.end] for note in pm.instruments[4].notes])
notes = np.array([note.pitch for note in pm.instruments[4].notes])

URI_notes_aligned_output = os.path.join('/Users/joro/Downloads/','alignedNotes.txt') # convetion name for this repo 

f = open(URI_notes_aligned_output,'wb')
writer = csv.writer(f, delimiter='\t')
work_notes = notes

for interval, note in zip(intervals, notes): # read intervals

    row = [interval[0]]
    row.append(interval[1]-interval[0])
    row.append(note)
    row.append(0)

    writer.writerow(row)
    
f.close()
print 'written file ' + URI_notes_aligned_output

written file /Users/joro/Downloads/alignedNotes.txt
